# Setup

In [1]:
import os

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://dalle3-swo.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "e51119f8d8774069a6594d92ccf7a70d"

In [2]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_core.output_parsers import BaseOutputParser
from typing import List, Optional
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo",
    temperature=0
) 
smart_llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-4",
    temperature=0
) 
gpt_35_turbo_instruct = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-instruct",
    temperature=0
) 
# Embedding
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2024-02-15-preview",
)

# Current Problems

bug:
- hiện thành table của df markdown
- 1 số tên lecturer ko tìm thấy thông tin
- hi, im John, please check my calendar .... -> ko tìm ra

# Test questions

In [9]:
question_lists = [
    "Hi, I'm Jon Neale, Please check my calender",
    "Hi, I'm David Holloway, tell me learning timetable of 3D game subject",
    "I'm Darius Postma, tell me my timetable for this friday",
    "I'm Darius Postma, tell me my timetable for this monday",
    "Hi, I'm David Holloway, please check if any lecture that i need to teach"
]

# Implementation

## Process input

In [30]:
import pandas as pd

df = pd.read_excel("../data/timetable/POC_TimeTableData.xlsx")
df["DAY_OF_WEEK"] = df["Date"].dt.day_name()

In [31]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///timetable.db")
df.to_sql("timetable", engine, index=False)

173

## Create agemt

In [5]:
from langchain_community.agent_toolkits import create_sql_agent
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase

engine = create_engine("sqlite:///timetable.db")
db = SQLDatabase(engine=engine, sample_rows_in_table_info=2)
agent_executor = create_sql_agent(smart_llm, db=db, agent_type="openai-tools", verbose=True)

In [22]:
question_lists

["Hi, I'm Jon Neale, Please check my calender",
 'tell me learning timetable of 3D game subject',
 "I'm Darius Postma, tell me my timetable for this friday",
 "I'm Darius Postma, tell me my timetable for this monday",
 "Hi, I'm David Holloway, please check if any lecture that i need to teach"]

In [17]:
agent_executor.invoke({"input": question_lists[0]})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


timetable
Invoking: `sql_db_schema` with `{'table_names': 'timetable'}`



CREATE TABLE timetable (
	"Date" DATETIME, 
	"Location" TEXT, 
	"Start Time" TIME, 
	"End Time" TIME, 
	"Module" TEXT, 
	"Class Size" BIGINT, 
	"Hour(s)" TIME, 
	"Student Set(s)" TEXT, 
	"Lecturer" TEXT, 
	"Note" TEXT, 
	"Students" TEXT
)

/*
3 rows from timetable table:
Date	Location	Start Time	End Time	Module	Class Size	Hour(s)	Student Set(s)	Lecturer	Note	Students
2024-04-22 00:00:00	Building B. 1-4 Computer Lab	12:00:00	14:00:00	IT Infrastructure Security 2/LEC/1/1	16	02:00:00	2110SUCS-Y3S2/001	Hamza Mutaher	 	HAN21080387
2024-04-22 00:00:00	Building B. 2-12 BUV GamePAD	09:00:00	11:00:00	Introduction to 3D Games Engines 2/LEC/1/1	23	02:00:00	2310SUCGDP-Y1S2/001	David Holloway,Quy Nguyen (TA)	 	HAN23080286
2024-04-22 00:00:00	Building B. 2-12 BUV GamePAD	11:00:00	12:00:00	Introduction to 3D Games Engines 2/TUT/1/1	23	01:0

{'input': "Hi, I'm Jon Neale, Please check my calender",
 'output': 'Here are the upcoming events on your calendar, Jon Neale:\n\n1. Date: 2024-04-26, Time: 15:00-17:00, Module: Professional Development (Enterprise, Entrepreneurship and Citizenship)/TUT/5/1, Location: Building B. 3-12 Multipurpose Hall\n2. Date: 2024-04-25, Time: 15:00-17:00, Module: Professional Development (Enterprise, Entrepreneurship and Citizenship)/TUT/4/1, Location: Building B. 3-12 Multipurpose Hall\n3. Date: 2024-04-24, Time: 15:00-17:00, Module: Professional Development (Enterprise, Entrepreneurship and Citizenship)/TUT/3/1, Location: Building B. 3-12 Multipurpose Hall\n4. Date: 2024-04-23, Time: 15:00-17:00, Module: Professional Development (Enterprise, Entrepreneurship and Citizenship)/TUT/2/1, Location: Building B. 3-12 Multipurpose Hall\n5. Date: 2024-04-22, Time: 15:00-17:00, Module: Professional Development (Enterprise, Entrepreneurship and Citizenship)/TUT/1/1, Location: Building B. 3-12 Multipurpose H

In [10]:
agent_executor.invoke({"input": question_lists[1]})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


timetable
Invoking: `sql_db_schema` with `{'table_names': 'timetable'}`



CREATE TABLE timetable (
	"Date" DATETIME, 
	"Location" TEXT, 
	"Start Time" TIME, 
	"End Time" TIME, 
	"Module" TEXT, 
	"Class Size" BIGINT, 
	"Hour(s)" TIME, 
	"Student Set(s)" TEXT, 
	"Lecturer" TEXT, 
	"Note" TEXT, 
	"Students" TEXT, 
	"DAY_OF_WEEK" TEXT
)

/*
2 rows from timetable table:
Date	Location	Start Time	End Time	Module	Class Size	Hour(s)	Student Set(s)	Lecturer	Note	Students	DAY_OF_WEEK
2024-04-22 00:00:00	Building B. 1-4 Computer Lab	12:00:00	14:00:00	IT Infrastructure Security 2/LEC/1/1	16	02:00:00	2110SUCS-Y3S2/001	Hamza Mutaher	 	HAN21080387	Monday
2024-04-22 00:00:00	Building B. 2-12 BUV GamePAD	09:00:00	11:00:00	Introduction to 3D Games Engines 2/LEC/1/1	23	02:00:00	2310SUCGDP-Y1S2/001	David Holloway,Quy Nguyen (TA)	 	HAN23080286	Monday
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT Date, Sta

{'input': "Hi, I'm David Holloway, tell me learning timetable of 3D game subject",
 'output': 'The timetable for the 3D game subject taught by David Holloway is as follows:\n\n1. Date: April 22, 2024, Start Time: 09:00, End Time: 11:00, Location: Building B. 2-12 BUV GamePAD\n2. Date: April 22, 2024, Start Time: 11:00, End Time: 12:00, Location: Building B. 2-12 BUV GamePAD'}

In [23]:
agent_executor.invoke({"input": question_lists[2]})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


timetable
Invoking: `sql_db_schema` with `{'table_names': 'timetable'}`



CREATE TABLE timetable (
	"Date" DATETIME, 
	"Location" TEXT, 
	"Start Time" TIME, 
	"End Time" TIME, 
	"Module" TEXT, 
	"Class Size" BIGINT, 
	"Hour(s)" TIME, 
	"Student Set(s)" TEXT, 
	"Lecturer" TEXT, 
	"Note" TEXT, 
	"Students" TEXT
)

/*
3 rows from timetable table:
Date	Location	Start Time	End Time	Module	Class Size	Hour(s)	Student Set(s)	Lecturer	Note	Students
2024-04-22 00:00:00	Building B. 1-4 Computer Lab	12:00:00	14:00:00	IT Infrastructure Security 2/LEC/1/1	16	02:00:00	2110SUCS-Y3S2/001	Hamza Mutaher	 	HAN21080387
2024-04-22 00:00:00	Building B. 2-12 BUV GamePAD	09:00:00	11:00:00	Introduction to 3D Games Engines 2/LEC/1/1	23	02:00:00	2310SUCGDP-Y1S2/001	David Holloway,Quy Nguyen (TA)	 	HAN23080286
2024-04-22 00:00:00	Building B. 2-12 BUV GamePAD	11:00:00	12:00:00	Introduction to 3D Games Engines 2/TUT/1/1	23	01:0

{'input': "I'm Darius Postma, tell me my timetable for this friday",
 'output': "I'm sorry, but it seems there are no classes scheduled for Darius Postma this Friday."}

In [8]:
agent_executor.invoke({"input": question_lists[3]})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


timetable
Invoking: `sql_db_schema` with `{'table_names': 'timetable'}`



CREATE TABLE timetable (
	"Date" DATETIME, 
	"Location" TEXT, 
	"Start Time" TIME, 
	"End Time" TIME, 
	"Module" TEXT, 
	"Class Size" BIGINT, 
	"Hour(s)" TIME, 
	"Student Set(s)" TEXT, 
	"Lecturer" TEXT, 
	"Note" TEXT, 
	"Students" TEXT, 
	"DAY_OF_WEEK" TEXT
)

/*
2 rows from timetable table:
Date	Location	Start Time	End Time	Module	Class Size	Hour(s)	Student Set(s)	Lecturer	Note	Students	DAY_OF_WEEK
2024-04-22 00:00:00	Building B. 1-4 Computer Lab	12:00:00	14:00:00	IT Infrastructure Security 2/LEC/1/1	16	02:00:00	2110SUCS-Y3S2/001	Hamza Mutaher	 	HAN21080387	Monday
2024-04-22 00:00:00	Building B. 2-12 BUV GamePAD	09:00:00	11:00:00	Introduction to 3D Games Engines 2/LEC/1/1	23	02:00:00	2310SUCGDP-Y1S2/001	David Holloway,Quy Nguyen (TA)	 	HAN23080286	Monday
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT Date, Sta

{'input': "I'm Darius Postma, tell me my timetable for this monday",
 'output': 'On Monday, Darius Postma has the following schedule:\n\n- Exploring the Eventscape 1/LEC/1/1 from 09:00 to 11:00 at Building B. 3-5 Learning Studio.'}

In [18]:
agent_executor.invoke({"input": question_lists[4]})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


timetable
Invoking: `sql_db_schema` with `{'table_names': 'timetable'}`



CREATE TABLE timetable (
	"Date" DATETIME, 
	"Location" TEXT, 
	"Start Time" TIME, 
	"End Time" TIME, 
	"Module" TEXT, 
	"Class Size" BIGINT, 
	"Hour(s)" TIME, 
	"Student Set(s)" TEXT, 
	"Lecturer" TEXT, 
	"Note" TEXT, 
	"Students" TEXT
)

/*
3 rows from timetable table:
Date	Location	Start Time	End Time	Module	Class Size	Hour(s)	Student Set(s)	Lecturer	Note	Students
2024-04-22 00:00:00	Building B. 1-4 Computer Lab	12:00:00	14:00:00	IT Infrastructure Security 2/LEC/1/1	16	02:00:00	2110SUCS-Y3S2/001	Hamza Mutaher	 	HAN21080387
2024-04-22 00:00:00	Building B. 2-12 BUV GamePAD	09:00:00	11:00:00	Introduction to 3D Games Engines 2/LEC/1/1	23	02:00:00	2310SUCGDP-Y1S2/001	David Holloway,Quy Nguyen (TA)	 	HAN23080286
2024-04-22 00:00:00	Building B. 2-12 BUV GamePAD	11:00:00	12:00:00	Introduction to 3D Games Engines 2/TUT/1/1	23	01:0

{'input': "Hi, I'm David Holloway, please check if any lecture that i need to teach",
 'output': 'Here are the lectures you need to teach:\n\n1. Date: 2024-04-22, Start Time: 09:00:00, End Time: 11:00:00, Module: Introduction to 3D Games Engines 2/LEC/1/1, Location: Building B. 2-12 BUV GamePAD\n2. Date: 2024-04-22, Start Time: 11:00:00, End Time: 12:00:00, Module: Introduction to 3D Games Engines 2/TUT/1/1, Location: Building B. 2-12 BUV GamePAD\n3. Date: 2024-04-23, Start Time: 09:00:00, End Time: 11:00:00, Module: Games Engine Creation 2/LEC/1/1, Location: Building B. 1-4 Computer Lab\n4. Date: 2024-04-23, Start Time: 11:00:00, End Time: 12:00:00, Module: Games Engine Creation 2/TUT/1/1, Location: Building B. 1-4 Computer Lab'}

In [19]:
agent_executor_test = create_sql_agent(gpt_35_turbo_instruct, db=db, agent_type="openai-tools", verbose=True)

In [20]:
agent_executor_test.invoke({"input": question_lists[3]})



> Entering new SQL Agent Executor chain...


BadRequestError: Error code: 400 - {'error': {'code': 'OperationNotSupported', 'message': 'The chatCompletion operation does not work with the specified model, gpt-35-turbo-instruct. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.'}}

# Testing

In [24]:
bot_answers = rag_chain.batch(question_lists)

In [26]:
import pandas as pd
answer_df = pd.DataFrame({
    "Question": question_lists,
    "Bot Answer": bot_answers
})

In [29]:
pd.set_option("display.max_colwidth", None)

In [ ]:
print(answer_df.to_markdown())

|    | Question                                                                                        | Bot Answer                                                                                                                                                              |
|---:|:------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | I'm at BUV. I want to go to 87 Lang Ha, tell me available bus                                   | From BUV Campus to 87 Lang Ha, the available bus slots are as follows:                                                                                                  |
|    |                                                                                                 | - Slot 1: Mon to Fri at 12:20                                                                                                                                           |
|    |                                                                                                 | - Slot 2: Mon to Fri at 13:20                                                                                                                                           |
|    |                                                                                                 | - Slot 3: Wed and Fri at 14:20                                                                                                                                          |
|    |                                                                                                 | - Slot 4: Mon to Fri at 16:05                                                                                                                                           |
|    |                                                                                                 | - Slot 5: Mon to Fri at 18:25                                                                                                                                           |
|    |                                                                                                 | Thanks for asking!                                                                                                                                                      |
|  1 | Tell me what are available routes for Monday 22 Apr                                             | On Monday 22 Apr, the available routes are:                                                                                                                             |
|    |                                                                                                 |                                                                                                                                                                         |
|    |                                                                                                 | For the Cau Giay Route:                                                                                                                                                 |
|    |                                                                                                 | - Pick-up from Nguyen Quoc Tri (Nam Trung Yen Building) at 07:00, 09:30, 11:40.                                                                                         |
|    |                                                                                                 | - Pick-up from 87 Lang Ha (National Cinema Centre) at 07:15, 09:45, 11:55.                                                                                              |
|    |                                                                                                 | - Drop-off at BUV Campus at 11:30, 12:30, 15:15, 17:35.                                                                                                                 |
|    |                                                                                                 | - Drop-off at 87 Lang Ha (National Cinema Centre) at 12:20, 13:20, 16:05, 18:25.                                                                                        |
|    |                                                                                                 | - Drop-off at Nguyen Quoc Tri (Nam Trung Yen Building) at 12:35, 13:35, 16:20, 18:40.                                                                                   |
|    |                                                                                                 |                                                                                                                                                                         |
|    |                                                                                                 | For the Tay Ho Route:                                                                                                                                                   |
|    |                                                                                                 | - Pick-up from 107 Xuan Dieu (In front of Apollo English Centre) at 07:40, 11:40.                                                                                       |
|    |                                                                                                 | - Pick-up from Ho Tay Water Park at 07:50, 11:50.                                                                                                                       |
|    |                                                                                                 | - Pick-up from Vinhomes Ocean Park at 08:30, 12:30.                                                                                                                     |
|    |                                                                                                 | - Drop-off at Vinhomes Ocean Park at 12:40, 15:25, 17:45.                                                                                                               |
|    |                                                                                                 | - Drop-off at Ho Tay Water Park at 13:20, 16:05, 18:25.                                                                                                                 |
|    |                                                                                                 | - Drop-off at 107 Xuan Dieu (In front of Apollo English Centre) at 13:30, 16:15, 18:35.                                                                                 |
|    |                                                                                                 |                                                                                                                                                                         |
|    |                                                                                                 | Thanks for asking!                                                                                                                                                      |
|  2 | Tell me what are available routes for Monday 08 Apr                                             | On Monday 08 Apr, the available routes are:                                                                                                                             |
|    |                                                                                                 |                                                                                                                                                                         |
|    |                                                                                                 | For the Cau Giay route:                                                                                                                                                 |
|    |                                                                                                 | - From Nguyen Quoc Tri (Nam Trung Yen Building) at 07:00 and 11:40.                                                                                                     |
|    |                                                                                                 | - From 87 Lang Ha (National Cinema Centre) at 07:15 and 11:55.                                                                                                          |
|    |                                                                                                 | - From BUV Campus at 08:05 and 12:45.                                                                                                                                   |
|    |                                                                                                 |                                                                                                                                                                         |
|    |                                                                                                 | For the Tay Ho route:                                                                                                                                                   |
|    |                                                                                                 | - From 107 Xuan Dieu (In front of Apollo English Centre) at 07:40 and 11:40.                                                                                            |
|    |                                                                                                 | - From Ho Tay Water Park at 07:50 and 11:50.                                                                                                                            |
|    |                                                                                                 | - From Vinhomes Ocean Park at 08:30 and 12:30.                                                                                                                          |
|    |                                                                                                 | - From BUV Ecopark at 08:40 and 12:40.                                                                                                                                  |
|    |                                                                                                 |                                                                                                                                                                         |
|    |                                                                                                 | Thanks for asking!                                                                                                                                                      |
|  3 | Can you tell me bus schedule from BUV to Lang ha                                                | The bus schedule from BUV Campus to Lang Ha (National Cinema Centre) is as follows:                                                                                     |
|    |                                                                                                 |                                                                                                                                                                         |
|    |                                                                                                 | - Slot 1: 12:20 (Mon to Fri)                                                                                                                                            |
|    |                                                                                                 | - Slot 2: 13:20 (Mon to Fri)                                                                                                                                            |
|    |                                                                                                 | - Slot 3: 14:20 (Wed and Fri)                                                                                                                                           |
|    |                                                                                                 | - Slot 4: 16:05 (Mon to Fri)                                                                                                                                            |
|    |                                                                                                 | - Slot 5: 18:25 (Mon to Fri)                                                                                                                                            |
|    |                                                                                                 |                                                                                                                                                                         |
|    |                                                                                                 | Thanks for asking!                                                                                                                                                      |
|  4 | Is there a bus available from BUV CAMPUS to CAU GIAY after my last class at 15:30?              | Yes, there are two buses available from BUV Campus to Cau Giay after your class at 15:30. The first one leaves at 15:15 and the second one at 17:35. Thanks for asking! |
|  5 | What is the last bus timing from 87 LANG HA to BUV CAMPUS? I have a study group that ends late. | The last bus from 87 LANG HA (National Cinema Centre) to BUV CAMPUS is at 13:45. Thanks for asking!                                                                     |
|  6 | What is the last bus timing from 87 LANG HA to BUV CAMPUS on Wednesdays?                        | The last bus timing from 87 LANG HA (National Cinema Centre) to BUV CAMPUS on Wednesdays is at 13:45. Thanks for asking!                                                |
|  7 | How do I get updated information about daily traffic situations for the CAU GIAY route?         | You can get updated information about the daily traffic situation for the Cau Giay route by scanning the provided QR code. Thanks for asking!                           |
